# 프롬프트 엔지니어링 시작하기
by DAIR.AI | 엘비스 사라비아


이 노트북에는 프롬프트 엔지니어링에 대해 배울 수 있는 예제와 연습 문제가 포함되어 있습니다.

모든 예제에서는 [OpenAI API](https://platform.openai.com/)를 사용합니다. 기본 설정인 'temperature=0.7'과 'top-p=1'을 사용하고 있습니다.

---

## 1. 프롬프트 엔지니어링 기본 사항

목표
- 라이브러리 로드
- 형식 검토하기
- 기본 프롬프트 다루기
- 일반적인 사용 사례 검토

아래에서 필요한 라이브러리, 유틸리티 및 구성을 로드하고 있습니다.

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv

In [3]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

환경 변수를 로드합니다. 원하는 것을 사용할 수 있지만 저는 `python-dotenv`를 사용했습니다. OPENAI_API_KEY`로 `.env` 파일을 생성한 다음 로드하면 됩니다.

In [5]:
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [10]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, messages):
    """ GET completion from openai api"""

    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

기본 프롬프트 예제:

In [11]:
# basic example
params = set_open_params()

prompt = "The sky is"

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [16]:
response.choices[0].message.content

'blue.'

다른 온도로 시도하여 결과를 비교하세요:

In [17]:
params = set_open_params(temperature=0)
response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

blue during the day and black at night.

### 1.1 텍스트 요약

In [18]:
params = set_open_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance. 

Explain the above in one sentence:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Antibiotics are medications that treat bacterial infections by either killing the bacteria or stopping their reproduction, but they are ineffective against viral infections and misuse can lead to antibiotic resistance.

연습하기: 모델에게 "나는 5살이다"와 같이 한 문장으로 단락을 설명하도록 지시하세요. 차이점이 보이나요?

### 1.2 질문 답변

In [19]:
prompt = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: What was OKT3 originally sourced from?

Answer:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


Mice.

여기에서 얻은 컨텍스트: https://www.nature.com/articles/d41586-023-00400-x

연습합니다: 프롬프트를 수정하고 모델에게 답이 확실하지 않다고 응답하도록 합니다.

### 1.3 텍스트 분류

In [21]:
prompt = """Classify the text into neutral, negative or positive.

Text: I think the food was okay.

Sentiment:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Neutral

연습: 모델에게 선택한 답에 대한 설명을 제공하도록 안내하는 프롬프트를 수정합니다.

### 1.4 롤 플레잉

In [22]:
prompt = """The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

messages = [
    {
        "role": "user",
        "content": prompt
    }

]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Certainly! Black holes are formed from the remnants of massive stars that have exhausted their nuclear fuel and undergone a supernova explosion. During this explosion, the outer layers of the star are blown away, leaving behind a dense core known as a stellar remnant. If the mass of the stellar remnant is above a certain threshold, called the Tolman-Oppenheimer-Volkoff limit, gravity becomes so strong that the core collapses in on itself, forming a black hole. This collapse is driven by the inward pull of gravity, and it results in a region of space where the gravitational field is so strong that nothing, not even light, can escape from it. This region is known as the event horizon.

연습: AI 응답을 간결하고 짧게 유지하도록 모델에 지시하도록 프롬프트를 수정합니다.

### 1.5 코드 생성

In [23]:
prompt = "\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n\"\"\""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


SELECT StudentName
FROM students
WHERE DepartmentId = (SELECT DepartmentId FROM departments WHERE DepartmentName = 'Computer Science')

### 1.6 추론

In [24]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

To solve this problem, we need to follow these steps:

Step 1: Identify the odd numbers in the given group. The odd numbers in the group are 15, 5, 13, 7, and 1.

Step 2: Add the odd numbers together. 15 + 5 + 13 + 7 + 1 = 41.

Step 3: Determine whether the sum is odd or even. In this case, the sum is 41, which is an odd number.

Therefore, the sum of the odd numbers in the given group is an odd number.

연습: 프롬프트의 구조와 출력 형식을 개선하여 더 나은 구조로 개선하세요.

## 2. 고급 프롬프트 기법

목표:

- 프롬프트에 대한 고급 기법(숏샷, 생각의 연쇄, ...)을 다룹니다.

### 2.2 몇 장 찍기 프롬프트

In [25]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

The answer is False.

2.3 생각의 연쇄(CoT) 프롬프트 ###

In [26]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.

### 2.4 제로샷 CoT

In [28]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Step 1: Bought 10 apples.
Step 2: Gave 2 apples to the neighbor and 2 apples to the repairman.
Remaining apples: 10 - 2 - 2 = 6 apples.
Step 3: Bought 5 more apples.
Total apples now: 6 + 5 = 11 apples.
Step 4: Ate 1 apple.
Remaining apples: 11 - 1 = 10 apples.

Final answer: You remained with 10 apples.

### 2.5 자기 일관성 유지
연습 삼아 [가이드](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#self-consistency)에서 예시를 확인하고 여기에서 시도해 보세요.

### 2.6 지식 프롬프트 생성하기

연습용으로 [가이드](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#generated-knowledge-prompting)에서 예시를 확인하고 여기에서 시도해 보세요.

---